In [ ]:
!pip install -U torchtext==0.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.1 MB 16.5 MB/s 
     |████████████████████████████████| 735.5 MB 14 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.


In [ ]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.3 MB/s eta 0:14:12tcmalloc: large alloc 1147494400 bytes == 0x38ed6000 @  0x7f153459a615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████               | 1055.7 MB 1.3 MB/s eta 0:12:09tcmalloc: large alloc 1434370048 bytes == 0x7d52c000 @  0x7f153459a615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

In [ ]:
import spacy
import random
import torch
import torch.nn as nn
import pandas as pd
import torchtext
from nltk.translate.bleu_score import sentence_bleu
from torchtext.vocab import build_vocab_from_iterator
from torchtext.legacy.data import Field, BucketIterator, TabularDataset
from sklearn.model_selection import train_test_split
import torch.optim as optim

In [1]:
torch.__version__

NameError: ignored

In [ ]:
!mkdir data

In [ ]:
batch_size = 64
num_layers = 2
hidden_size = 1024
embedding_size = 300
dropout_prob = 0.1
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size, embedding_size, dropour_prob, device):
        super(Encoder, self).__init__()
        self.device = device
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size

        self.embedding_layer = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, dropout=dropour_prob)

        for m in self.modules():
            if isinstance(m, nn.LSTM):
                for p in m.parameters():
                    p.data.uniform_(-0.08, 0.08)

    def forward(self, x):
        x = self.embedding_layer(x)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=self.device)
        out, (hidden, cell) = self.rnn(x, (h0, c0))
        return out, hidden, cell


In [ ]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, embedding_size, num_layers, vocab_size, dropout_p, device):
        super(Decoder, self).__init__()

        self.device = device
        self.vocab_size = vocab_size
        self.embedding_layer = nn.Embedding(vocab_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, dropout=dropout_p)
        self.fc = nn.Linear(hidden_size, vocab_size)

        for m in self.modules():
            if isinstance(m, nn.Linear) or isinstance(m, nn.LSTM):
                for p in m.parameters():
                    p.data.uniform_(-0.08, 0.08)

    def forward(self, x, hidden, cell):
        # x of shape (N, 1) as we pass 1 word at tume to our Decoder
        x = self.embedding_layer(x)
        output, (hidden_next, cell_next) = self.rnn(x, (hidden, cell))
        x_next = self.fc(output)
        return x_next, hidden_next, cell_next
 

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, source, target, target_word_ratio=0.5):
        seq_len = source.size(1)
        targ_len = target.size(1)
        output, hidden, cell = self.encoder(source)
        outputs = torch.zeros(source.size(0), targ_len, self.decoder.vocab_size,
                              device=self.device)
        x = source[:, 0].unsqueeze(1)

        for i in range(1, targ_len):
            x, hidden, cell = self.decoder(x, hidden, cell)
            outputs[:, i] = x.squeeze(1)
            preds = x.argmax(dim=-1).to(device=self.device)
            x = target[:, i].unsqueeze(1) if random.random() < target_word_ratio else preds

        return outputs

In [ ]:
class EncoderAttention(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size, embedding_size, dropour_prob, device):
        super(EncoderAttention, self).__init__()
        self.device = device
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size

        self.dropout = nn.Dropout(dropour_prob)
        self.embedding_layer = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, 
                           bidirectional=True)
        self.fc_hidden = nn.Linear(2*hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc_hidden = nn.Linear(2*hidden_size, hidden_size)
        self.fc_cell = nn.Linear(2*hidden_size, hidden_size)

        for m in self.modules():
            if isinstance(m, nn.LSTM):
                for p in m.parameters():
                    p.data.uniform_(-0.08, 0.08)

    def forward(self, x):
        x = self.dropout(self.embedding_layer(x))
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size, device=self.device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size, device=self.device)
        encoder_states, (hidden, cell) = self.rnn(x, (h0, c0))
        hidden = self.fc_hidden(torch.cat([hidden[0], hidden[1]], dim=-1))
        cell = self.fc_cell(torch.cat([cell[0], cell[1]], dim=-1))
        return encoder_states, hidden, cell

In [ ]:
class DecoderAttention(nn.Module):
    def __init__(self, hidden_size, embedding_size, num_layers, vocab_size, dropout_p, device):
        super(DecoderAttention, self).__init__()

        self.device = device
        self.vocab_size = vocab_size
        self.dropout = nn.Dropout(dropout_p)
        self.embedding_layer = nn.Embedding(vocab_size, embedding_size)
        self.rnn = nn.LSTM(2*hidden_size + embedding_size, hidden_size, num_layers, batch_first=True, dropout=dropout_p)
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.energy_layer = nn.Linear(3*hidden_size, 1)
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()

        for m in self.modules():
            if isinstance(m, nn.Linear) or isinstance(m, nn.LSTM):
                for p in m.parameters():
                    p.data.uniform_(-0.08, 0.08)

    def forward(self, encoder_states, x, hidden, cell):
        seq_len = encoder_states.size(1)
        batch_s = x.size(0)
        # x of shape (N, 1) as we pass 1 word at tume to our Decoder
        x = self.dropout(self.embedding_layer(x))
        hidden_reshaped = hidden.unsqueeze(1).repeat(1, seq_len, 1)
        mutual_state = torch.cat([encoder_states, hidden_reshaped], dim=-1).reshape((-1, hidden_size*3))
        energy = self.relu(self.energy_layer(mutual_state).reshape((batch_s, seq_len)))
        attention = self.softmax(energy).unsqueeze(-1)
        weighted_encoder_states = encoder_states * attention
        c_i = torch.sum(weighted_encoder_states, dim=1).unsqueeze(1)
        x_concat = torch.cat([c_i, x], dim=-1)
        output, (hidden_next, cell_next) = self.rnn(x_concat, (hidden.unsqueeze(0), cell.unsqueeze(0)))

        predictions = self.fc(output)
        hidden_next = hidden_next.squeeze(0)
        cell_next = cell_next.squeeze(0)
        return predictions, hidden_next, cell_next

In [ ]:
class Seq2SeqAttention(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2SeqAttention, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, source, target, target_word_ratio=0.5):
        seq_len = source.size(1)
        targ_len = target.size(1)
        encoder_states, hidden, cell = self.encoder(source)
        outputs = torch.zeros(source.size(0), targ_len, self.decoder.vocab_size,
                              device=self.device)
        x = source[:, 0].unsqueeze(1)

        for i in range(1, targ_len):
            x, hidden, cell = self.decoder(encoder_states, x, hidden, cell)
            outputs[:, i] = x.squeeze(1)
            preds = x.argmax(dim=-1).to(device=self.device)
            x = target[:, i].unsqueeze(1) if random.random() < target_word_ratio else preds

        return outputs

In [ ]:
encoder_attention = EncoderAttention(input_size_eng, 1, hidden_size, embedding_size, dropout_prob, device).to(device=device)
decoder_attention = DecoderAttention(hidden_size, embedding_size, 1, input_size_ger, dropout_prob, device).to(device)
seq2seq_attention = Seq2SeqAttention(encoder_attention, decoder_attention, device).to(device)

num_epochs = 100
optimizer = optim.Adam(seq2seq_attention.parameters(), lr=learning_rate)
padding_index = german.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=padding_index)
losses = []

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
for e in range(num_epochs):
  for b in train_iterator:
    e = b.eng.to(device)
    g = b.ger.to(device)

    outputs = seq2seq_attention(e, g).permute(0, 2, 1)
    loss = criterion(outputs, g)
    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(losses[-1])

0.8043912053108215
0.7918741703033447
0.816533088684082
0.9293058514595032
0.8571704030036926
0.8790246248245239
0.8210350871086121
0.8029860258102417
1.107661247253418
0.7760710716247559
0.795376718044281
0.7642459869384766
0.9233747720718384
0.7997989058494568
0.8392578959465027
0.7891374826431274
0.8399292826652527
0.8871402740478516
0.8388465046882629
0.9796174764633179
0.8960346579551697
0.7913787364959717
0.7847344279289246
0.9109691381454468
0.8713207840919495
0.8186138272285461
0.7836823463439941
0.7857446074485779
0.89989173412323
0.8316026926040649
0.7951050400733948
0.79215407371521
0.8296125531196594
0.843471348285675
1.0320285558700562
0.7719306945800781
0.8421376943588257
0.8235607743263245
0.8682052493095398
0.8693881034851074
0.8243828415870667
0.8052978515625
0.7760056853294373
0.8034777045249939
0.7755343317985535
0.9851535558700562
0.8583587408065796
0.7826148271560669
0.878818154335022
0.8917586803436279
0.9237167835235596
0.827684760093689
0.8357776999473572
0.8475

KeyboardInterrupt: ignored

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 26.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44.4 MB 118 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_md')


In [ ]:
tok_eng = spacy.load('en_core_web_sm')
tok_ger = spacy.load('de_core_news_md')

def tokenizer_eng(txt):
    return [tok.text for tok in tok_eng.tokenizer(txt)][:-1]


def tokenizer_ger(txt):
    return [tok.text for tok in tok_ger.tokenizer(txt)][:-1]


english = Field(sequential=True, use_vocab=True, lower=True, tokenize=tokenizer_eng, batch_first=True, eos_token='<eos>', init_token='<sos>')
german = Field(sequential=True, use_vocab=True, lower=True, tokenize=tokenizer_ger, batch_first=True, init_token='<sos>', eos_token='<eos>')
fields = {'English': ('eng', english), 'German': ('ger', german)}
train_data, test_data = TabularDataset.splits(
    path='data/',
    train='train_cleaned.csv',
    test='test_cleaned.csv',
    format='csv',
    fields=fields
)


english.build_vocab(train_data, max_size=10000, min_freq=2)
german.build_vocab(train_data, max_size=10000, min_freq=2)

train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data),
    batch_size=batch_size,
    device=device,
    sort=False
)

input_size_eng = len(english.vocab)
input_size_ger = len(german.vocab)
print(input_size_eng, input_size_ger)


5331 6707


In [ ]:
encoder = Encoder(input_size_eng, num_layers, hidden_size, embedding_size, dropout_prob, device).to(device)
decoder = Decoder(hidden_size, embedding_size, num_layers, input_size_ger, dropout_prob, device).to(device)
model = Seq2Seq(encoder, decoder, device).to(device)
num_epochs = 100
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
padding_index = german.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=padding_index)
losses = []

In [ ]:
for e in range(1):
  for b in train_iterator:
    english_sentences = b.eng.to(device=device)
    german_sentences = b.ger.to(device=device)
    outputs = model(english_sentences, german_sentences).permute(0, 2, 1)

    loss = criterion(outputs, german_sentences)
    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(losses[-1])

0.784904956817627
0.9042871594429016
1.0269476175308228
0.8764520287513733
0.8718395233154297
0.9632588028907776
0.8748547434806824
1.1845667362213135
0.7751920819282532
0.9738234281539917
1.013184905052185
0.9834968447685242
1.0793319940567017
0.8826574683189392
0.8394075632095337
0.8491674065589905
0.8159763813018799
0.8627195358276367
0.8162447810173035
0.9222586750984192
0.9426748752593994
0.878289520740509
0.8504347205162048
1.144720435142517
0.9881502985954285
0.8387945294380188
0.940334141254425
0.9122622609138489
0.7935078740119934
0.8315557837486267
0.93814617395401
0.9834305047988892
0.8088600039482117
0.8236036896705627
0.8251978158950806
0.8463721871376038
1.0805840492248535
0.8606700897216797
0.8160576820373535
0.9221278429031372
0.8199814558029175
0.8764519691467285
0.9540594816207886
0.8416733741760254
0.779752790927887
0.8332123756408691
0.837063729763031
0.8746078610420227
1.0502681732177734
1.014448642730713
1.161900281906128
0.8245198130607605
0.9733766913414001
0.88

In [ ]:
torch.save({
    'encoder': encoder.state_dict(),
    'decoder': decoder.state_dict(),
    'seq2seq': model.state_dict(),
    'optim': optimizer.state_dict()
}, 'state.pt')

In [ ]:
encoder = encoder_attention
decoder = decoder_attention
model = seq2seq_attention

In [ ]:
encoder.eval()
decoder.eval()
model.eval()

Seq2SeqAttention(
  (encoder): EncoderAttention(
    (dropout): Dropout(p=0.1, inplace=False)
    (embedding_layer): Embedding(5331, 300)
    (rnn): LSTM(300, 1024, batch_first=True, bidirectional=True)
    (fc_hidden): Linear(in_features=2048, out_features=1024, bias=True)
    (relu): ReLU()
    (fc_cell): Linear(in_features=2048, out_features=1024, bias=True)
  )
  (decoder): DecoderAttention(
    (dropout): Dropout(p=0.1, inplace=False)
    (embedding_layer): Embedding(6707, 300)
    (rnn): LSTM(2348, 1024, batch_first=True, dropout=0.1)
    (fc): Linear(in_features=1024, out_features=6707, bias=True)
    (energy_layer): Linear(in_features=3072, out_features=1, bias=True)
    (softmax): Softmax(dim=1)
    (relu): ReLU()
  )
)

In [ ]:
encoder.train()
decoder.train()
model.train()

Seq2SeqAttention(
  (encoder): EncoderAttention(
    (dropout): Dropout(p=0.1, inplace=False)
    (embedding_layer): Embedding(5331, 300)
    (rnn): LSTM(300, 1024, batch_first=True, bidirectional=True)
    (fc_hidden): Linear(in_features=2048, out_features=1024, bias=True)
    (relu): ReLU()
    (fc_cell): Linear(in_features=2048, out_features=1024, bias=True)
  )
  (decoder): DecoderAttention(
    (dropout): Dropout(p=0.1, inplace=False)
    (embedding_layer): Embedding(6707, 300)
    (rnn): LSTM(2348, 1024, batch_first=True, dropout=0.1)
    (fc): Linear(in_features=1024, out_features=6707, bias=True)
    (energy_layer): Linear(in_features=3072, out_features=1, bias=True)
    (softmax): Softmax(dim=1)
    (relu): ReLU()
  )
)

In [ ]:
def bleu_score(reference, candidate, weights):
    return sentence_bleu(reference, candidate, weights=weights)

In [ ]:
def translate_first(eng_sentence):
  out, hidden, cell = encoder(eng_sentence)
  words_indexes = []
  x = torch.tensor([[german.vocab.stoi['<sos>']]]).to(device=device)
  while x != german.vocab.stoi['<eos>']:
    x, hidden, cell = decoder(x, hidden, cell)
    x = x.argmax(-1)
    words_indexes.append(x.item())
  return [german.vocab.itos[ind] for ind in words_indexes[:-1]]

In [ ]:
def translate(eng_sentence):
  out, hidden, cell = encoder(eng_sentence)
  words_indexes = []
  x = torch.tensor([[german.vocab.stoi['<sos>']]]).to(device=device)
  while x != german.vocab.stoi['<eos>']:
    x, hidden, cell = decoder(out, x, hidden, cell)
    x = x.argmax(-1)
    words_indexes.append(x.item())
  return [german.vocab.itos[ind] for ind in words_indexes[:-1]]

In [ ]:
def bleu_score_from_iterator(iterator, weights=(0.4, 0.4, 0.2, 0)):
  translated_sentences = []
  target_sentences = []

  for b in iterator:
    target_sent = b.ger
    for i in range(target_sent.shape[0]):
      sent = list(target_sent[i])
      target_sentences.append([german.vocab.itos[ind] for ind in sent[1:sent.index(3)]])

  for b in test_iterator:
    english_sent = b.eng.to(device=device)
    german_sent = b.ger.to(device=device)
    for i in range(english_sent.shape[0]):
      translated_sentences.append(translate(english_sent[i].unsqueeze(0)))
  
  bleu_sum_score = 0
  for pair in zip(target_sentences, translated_sentences):
    bleu_sum_score += bleu_score([pair[0]], pair[1], weights=weights)
  return bleu_sum_score / len(translated_sentences)
  

In [ ]:
bleu_score_from_iterator(test_iterator)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

0.25537310452808687

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg